# Modeling a Simultaneous Multi-Round Auction with `Z3`

This notebook models the simultaneous multi-round auction from the paper ["Analyzing and Predicting Verification of
Data-Aware Process Models—A Case
Study With Spectrum Auctions"](https://doi.org/10.1109/ACCESS.2022.3154445) as an SMT problem.
As solver (= optimizer), we use the theorem prover [`Z3`](https://github.com/Z3Prover/z3).
The original verification procedure involved modeling the auction with BPMN (including data values), reducing the BPMN models, transforming to Petri nets, and verifying about 130k properties with the model checker `LoLA`.
Overall, the verification took several days.
In contrast, we only need one optimizer call here, which finishes in less than a second 🙃

In [1]:
import numpy as np
import z3

import time

## Constants

In general, a simultaneous multi-round auction has $m$ bidders and $n$ products.
We don't model the multiple rounds, as our objective and constraints all relate to the result after the final round.
We give the variables and constants in the code descriptive names, but use the shorthand notation in formulas in the explanatory text.
We use the index $1 \leq i \leq m$ for bidders and the index $1 \leq j \leq n$ for products.
The concrete auction we model has 4 bidders and 6 products.
The auction design resembles the German 4G spectrum auction for the 800 MHz band:

In [2]:
NUM_BIDDERS = 4
NUM_PRODUCTS = 6

Each bidder has a budget $b_{ij}$ for each product, which we define as in the paper linked above:

In [3]:
budgets = [[90, 90, 90, 60, 90, 90],
           [90, 80, 80, 80, 80, 80],
           [70, 80, 70, 70, 70, 60],
           [60, 60, 70, 60, 90, 60]]

Each bidder also has a capacity $c_i$, i.e., a maximum number of products they are allowed to win: 

In [4]:
capacities = [2, 3, 2, 1]

Each product has a reserve price $r_j$, i.e., a minimum price:

In [5]:
reserve_prices = [3, 3, 3, 3, 3, 3]

## Variables

There are two kinds of variables.

First, let $x_{ij} \in \{0,1\}$ denote whether Bidder $i$ wins Product $j$:

In [6]:
is_winner = [[z3.Bool(f'x_{i},{j}') for j in range(NUM_PRODUCTS)] for i in range(NUM_BIDDERS)]

Second, let $p_j \in \mathbb{R}$ denote the final price of each product:

In [7]:
prices = [z3.Real(f'p_{j}') for j in range(NUM_PRODUCTS)]

## Objective

We are ready to create the instance of the optimization problem:

In [8]:
optimizer = z3.Optimize()

Our objective is to determine the minimum revenue for the auctioneer, i.e., $ \min \sum_{j=1}^n p_j$:

In [9]:
objective = optimizer.minimize(z3.Sum(prices))

## Constraints

There are five kinds of constraints.

First, the final price of each product has to be at least its corresponding reserve price, i.e., $\forall~1 \leq j \leq n:~p_j \geq r_j$:

In [10]:
for j in range(NUM_PRODUCTS):
    optimizer.add(prices[j] >= reserve_prices[j])

Second, each product has to be won by exactly one bidder, i.e., $\forall~1 \leq j \leq n:~\sum_{i=1}^m x_{ij} = 1$:

In [11]:
for j in range(NUM_PRODUCTS):
    optimizer.add(z3.AtLeast(*[is_winner[i][j] for i in range(NUM_BIDDERS)], 1))
    optimizer.add(z3.AtMost(*[is_winner[i][j] for i in range(NUM_BIDDERS)], 1))

Third, each bidder has to stay within their capacity, i.e., $\forall~1 \leq i \leq m:~\sum_{j=1}^n x_{ij} \leq c_i$:

In [12]:
for i in range(NUM_BIDDERS):
    optimizer.add(z3.AtMost(*is_winner[i], capacities[i]))

Fourth, each bidder has to stay within their corresponding budget for each product they win, i.e., $\forall~1 \leq i \leq m~\forall~1 \leq j \leq n: x_{ij} \rightarrow (p_j \leq b_{ij})$:

In [13]:
for i in range(NUM_BIDDERS):
    for j in range(NUM_PRODUCTS):
        optimizer.add(z3.Implies(is_winner[i][j], prices[j] <= budgets[i][j]))

Fifth, if bidders don't use their full capacity, they must be out of budget for all products, i.e., $\forall~1 \leq i \leq m:~(\sum_{j=1}^n x_{ij} < c_i) \rightarrow (\bigwedge_{1 \leq j \leq n}~p_j \geq b_{ij})$.
This means that there must not be bidders that have both budget and capacity left (because in that case they would continue bidding):

In [14]:
for i in range(NUM_BIDDERS):
    optimizer.add(z3.Implies(z3.AtMost(*is_winner[i], capacities[i] - 1),
                             z3.And([p >= b for p, b in zip(prices, budgets[i])])))

As an alternative, one could also model the logical contrapositive, i.e., if bidders don't bid higher though they could, they must be out of capacity, i.e., $\forall~1 \leq i \leq m:~(\bigvee_{1 \leq j \leq n}~p_j < b_{ij}) \rightarrow (\sum_{j=1}^n x_{ij} = c_i)$:

In [15]:
# for i in range(NUM_BIDDERS):
#     optimizer.add(z3.Implies(z3.Or([p < b for p, b in zip(prices, budgets[i])]),
#                              z3.AtLeast(*is_winner[i], capacities[i])))

## Optimization

Now we have all ingredients to run the optimizer:

In [16]:
start_time = time.perf_counter()
result = optimizer.check()
end_time = time.perf_counter()
print('Result:', result)
print('Time:', round(end_time - start_time, 3), 's')

Result: sat
Time: 0.012 s


The problem is satisfiable, and the solver was quite fast (granted, the problem was very small).

Let's have a look at the objective and the variable assignments:

In [17]:
print('Revenue:', objective.value())
print('Prices:', [optimizer.model()[p] for p in prices])
is_winner_values = np.array([[int(bool(optimizer.model()[is_winner[i][j]]))
                              for j in range(NUM_PRODUCTS)] for i in range(NUM_BIDDERS)])
print('Winners:', np.argmax(is_winner_values, axis=0) + 1)  # "+ 1" to avoid zero-indexing

Revenue: 420
Prices: [70, 80, 70, 70, 70, 60]
Winners: [2 1 1 2 4 2]


The results look sensible, but for unknown reasons, the minimum revenue is 10 less than in the Petri-net modeling/verification.